# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [5]:
import torch
# lib optim
from torch import optim
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms # the library for transform of data
from tqdm import tqdm
from torch.autograd import Variable
import time

BATCH_SIZE = 128
# if u r using simple net, the num_epochs 10~100 have same results.
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [6]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):
        super(SimpleNet, self).__init__()
        self.layer1 = nn.Linear(in_dim, n_hidden_1)
        self.layer2 = nn.Linear(n_hidden_1, n_hidden_2)
        self.layer3 = nn.Linear(n_hidden_2, out_dim)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x
    
# model-2: add activate function:    
class Activation_Net(nn.Module):
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):
        super(Activation_Net, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(in_dim, n_hidden_1), nn.ReLU(True))
        self.layer2 = nn.Sequential(nn.Linear(n_hidden_1, n_hidden_2), nn.ReLU(True))
        self.layer3 = nn.Sequential(nn.Linear(n_hidden_2, out_dim))
 
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

# model-3: Add Batch Normalization:
class Batch_Net(nn.Module):
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):
        super(Batch_Net, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(in_dim, n_hidden_1), nn.BatchNorm1d(n_hidden_1), nn.ReLU(True))
        self.layer2 = nn.Sequential(nn.Linear(n_hidden_1, n_hidden_2), nn.BatchNorm1d(n_hidden_2), nn.ReLU(True))
        self.layer3 = nn.Sequential(nn.Linear(n_hidden_2, out_dim))
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x
    
# model = SimpleNet(28*28, 300, 100, 10)
# model = Activation_Net(28 * 28, 300, 100, 10)
model = Batch_Net(28 * 28, 300, 100, 10)

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-2)

Next, we can start to train and evaluate!

In [35]:


# train and evaluate
for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        # forward:
        images = images.view(images.size(0), -1)
        images = Variable(images)
        labels = Variable(labels)
        out = model(images)
        loss = criterion(out, labels)
        # backward: 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
# evaluate
# TODO:calculate the accuracy using traning and testing dataset
model.eval()
eval_loss = 0
eval_acc = 0
for images, labels in tqdm(test_loader):
    images = images.view(images.size(0),-1)
    images = Variable(images)
    labels = Variable(labels)
    out = model(images)
    loss = criterion(out, labels)
    eval_loss += loss.data.item()*labels.size(0)
    _, pred = torch.max(out, 1)
    num_correct = (pred == labels).sum()
    eval_acc += num_correct.item()


100%|██████████| 78/78 [00:01<00:00, 68.22it/s]


#### Q5:
Please print the training and testing accuracy.

In [36]:
print('Using model_1 we have: Test Loss: {:.6f}, Acc: {:.6f}'.format(
    eval_loss / (len(test_dataset)),
    eval_acc / (len(test_dataset))
))

Using model_1 we have: Test Loss: 0.286384, Acc: 0.918300


## Back-up cell: Using Model 2

In [41]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        # forward:
        images = images.view(images.size(0), -1)
        images = Variable(images)
        labels = Variable(labels)
        out = model(images)
        loss = criterion(out, labels)
        # backward: 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
# evaluate
# TODO:calculate the accuracy using traning and testing dataset
model.eval()
eval_loss = 0
eval_acc = 0
for images, labels in tqdm(test_loader):
    images = images.view(images.size(0),-1)
    images = Variable(images)
    labels = Variable(labels)
    out = model(images)
    loss = criterion(out, labels)
    eval_loss += loss.data.item()*labels.size(0)
    _, pred = torch.max(out, 1)
    num_correct = (pred == labels).sum()
    eval_acc += num_correct.item()
print('Using model_2 we have: Test Loss: {:.6f}, Acc: {:.6f}'.format(
    eval_loss / (len(test_dataset)),
    eval_acc / (len(test_dataset))
))

100%|██████████| 78/78 [00:01<00:00, 63.76it/s]

Test Loss: 0.246158, Acc: 0.926100


## Back-up cell: Using Model 3

In [7]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        # forward:
        images = images.view(images.size(0), -1)
        images = Variable(images)
        labels = Variable(labels)
        out = model(images)
        loss = criterion(out, labels)
        # backward: 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
# evaluate
# TODO:calculate the accuracy using traning and testing dataset
model.eval()
eval_loss = 0
eval_acc = 0
for images, labels in tqdm(test_loader):
    images = images.view(images.size(0),-1)
    images = Variable(images)
    labels = Variable(labels)
    out = model(images)
    loss = criterion(out, labels)
    eval_loss += loss.data.item()*labels.size(0)
    _, pred = torch.max(out, 1)
    num_correct = (pred == labels).sum()
    eval_acc += num_correct.item()
    
print('Using model_3 we have: Test Loss: {:.6f}, Acc on test_dateset: {:.6f}'.format(
    eval_loss / (len(test_dataset)),
    eval_acc / (len(test_dataset))
))

100%|██████████| 78/78 [00:01<00:00, 63.81it/s]

Using model_3 we have: Test Loss: 0.077905, Acc: 0.976700


In [10]:
model.eval()
eval_loss = 0
eval_acc = 0
for images, labels in tqdm(train_loader):
    images = images.view(images.size(0),-1)
    images = Variable(images)
    labels = Variable(labels)
    out = model(images)
    loss = criterion(out, labels)
    eval_loss += loss.data.item()*labels.size(0)
    _, pred = torch.max(out, 1)
    num_correct = (pred == labels).sum()
    eval_acc += num_correct.item()
    
print('Using model_3 we have: Test Loss: {:.6f}, Acc on training_dateset: {:.6f}'.format(
    eval_loss / (len(train_dataset)),
    eval_acc / (len(train_dataset))
))

100%|██████████| 468/468 [00:06<00:00, 69.41it/s]

Using model_3 we have: Test Loss: 0.044811, Acc on training_dateset: 0.989633


In [9]:
model.eval()
eval_loss = 0
eval_acc = 0
for images, labels in tqdm(test_loader):
    images = images.view(images.size(0),-1)
    images = Variable(images)
    labels = Variable(labels)
    out = model(images)
    loss = criterion(out, labels)
    eval_loss += loss.data.item()*labels.size(0)
    _, pred = torch.max(out, 1)
    num_correct = (pred == labels).sum()
    eval_acc += num_correct.item()
    
print('Using model_3 we have: Test Loss: {:.6f}, Acc on test_dateset: {:.6f}'.format(
    eval_loss / (len(test_dataset)),
    eval_acc / (len(test_dataset))
))

100%|██████████| 78/78 [00:01<00:00, 67.90it/s]

Using model_3 we have: Test Loss: 0.077905, Acc on test_dateset: 0.976700
